In [ ]:
%pip install ollama

In [29]:
import ollama

res = ollama.chat(model='llama3-ko', messages=[
    {
        'role': 'user',
        'content': '코사인의 미분 공식에 대해 설명해줘'
    }
])
print(res['message']['content'])

코사인 함수의 정의는 다음과 같습니다. 코사인은 사인 함수의 제곱입니다. 코사인 함수의 도함수는 다음과 같이 계산됩니다
